In [16]:
import csv
import pandas as pd
import numpy as np
import nltk
import re
import math
from unicodedata import normalize
from collections import Counter

In [17]:
leitura = pd.read_csv('../data/estadao_noticias_eleicao.csv')
leitura = leitura.replace(np.nan, '',regex = True)

In [18]:
def text_clear(text):
    pattern = re.compile('[^a-zA-Z0-9 ]')
    text = normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    return pattern.sub(' ', text)

In [19]:
conteudos = leitura.titulo + " " + leitura.subTitulo +  " " + leitura.conteudo
conteudos  = conteudos .apply(lambda text: "" if isinstance(text, float) else text_clear(text).lower())
ids = leitura.idNoticia

In [20]:
noticias = conteudos.apply(nltk.word_tokenize)
freq_term = noticias.apply(Counter)


In [22]:
index = {}
for i in range(len(noticias)):
    id_noticia = ids[i]
    for palavra in noticias[i]:
        palavra = palavra.lower()
        if palavra not in index:
            index[palavra] = {}
        id_rec = index[palavra].get(id_noticia)
        
        if not id_rec:
            docs = index[palavra]
            docs[id_noticia] = freq_term[i][palavra]

In [36]:
def generator_tf(phase):
    term = phase.split(' ')
    doc_tf = {}
    
    for i in range(len(term)):
        docs = index[term[i]]
        for doc_id in docs:
            tf = docs[doc_id]
            
            if doc_id not in doc_tf:
                doc_tf[doc_id] = np.array([0 if j != i else tf for j in range(len(term))])
            else:
                doc_vector = doc_tf[doc_id]
                doc_tf[doc_id] = np.array([doc_vector[j] if j != i else tf for j in range(len(term))])
    return doc_tf
    

In [49]:
def generator_idf(phase):
    terms = phase.split(' ')
    idf  = np.array([math.log((len(noticias)+1)/len(index[term])) for term in terms])
    return idf

In [41]:
def generator_query(phase):
    terms = phase.split(' ')
    query = np.array([1 if index.get(term) else 0 for term in terms])
    return query

In [58]:
def generato_bm25(phase):
    docs_tf = genarator_tf(phase)
    k = 5
    bm25_vetor = {doc_id: np.array([((k+1)*tf)/(tf+k) for tf in tf_vetor]) for doc_id, tf_vetor in docs_tf.items()}
    return bm25_vetor

In [44]:
def seach_tf(phase):
    docs_tf = generator_tf(phase)
    query = generator_query(phase)
    doc_rank = sorted(list(docs_tf.items()), key=lambda doc: np.dot(doc[1], query), reverse=True)[:5] 
    return [doc[0] for doc in doc_rank]

In [56]:
def seach_tf_idf(phase):
    doc_tf = generator_tf(phase)
    doc_idf = generator_idf(phase)
    doc_rank = sorted(list(doc_tf.items()), key=lambda doc: np.dot(doc[1], doc_idf), reverse=True)[:5]
    return [doc[0] for doc in doc_rank]
